In [1]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np

In [57]:
ee.Initialize()
L1_AGBPSeasonals = ee.ImageCollection("projects/fao-wapor/L1_AGBP")
ETaCollection = ee.ImageCollection("projects/fao-wapor/L1_AET")
AET250 = ee.ImageCollection("users/lpeiserfao/AET250")

# VETTORI
countries = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')
wsheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

In [3]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]

# Inizio calcolo WP

### Above Ground Biomass Production with masked NoData (pixel < 0) and summed

In [4]:
L1_AGBPSeasonalMasked =L1_AGBPSeasonals.map(lambda lista: lista.updateMask(lista.gte(0)))

#the multiplier will need to be applied on net FRAME delivery, not on sample dataset
L1_AGBPSummedYearly = L1_AGBPSeasonalMasked.sum(); #.multiply(10); 

### Actual Evapotranspiration with valid ETa values (>0 and <254)

In [5]:
ETaColl1 = AET250.map(lambda immagine: immagine.updateMask(immagine.lt(254) and (immagine.gt(0)))) 

### add image property (days in dekad) as band

In [6]:
ETaColl2 = ETaColl1.map(lambda immagine: immagine.addBands(immagine.metadata('days_in_dk')));

### get ET value, divide by 10 (as per FRAME spec) to get daily value, and multiply by number of days in dekad summed annually

In [7]:
ETaColl3 = ETaColl2.map(lambda immagine: immagine.select('b1').divide(10).multiply(immagine.select('days_in_dk'))).sum()

### scale ETsum from mm/m² to m³/ha for WP calculation purposes

In [8]:
ETaTotm3 = ETaColl3.multiply(10)

### calculate biomass water productivity and add to map

In [9]:
WPbm = L1_AGBPSummedYearly.divide(ETaTotm3)
WPbm.getInfo()

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   u'data_type': {u'precision': u'double', u'type': u'PixelType'},
   u'id': u'b1'}],
 u'type': u'Image'}

In [10]:
scala = WPbm.projection().nominalScale().getInfo()
visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region":region,
             "scale" : scala}

In [11]:
url_WPbm = WPbm.getThumbUrl(visparams)
Image(url=url_WPbm)

In [32]:
url_WPbm

u'https://earthengine.googleapis.com/api/thumb?thumbid=ca7e0513f35ed349ace23502091ac682&token=322308fd4c78ee187d7d73fbc52f9322'

In [43]:
mapid = WPbm.getMapId()

In [44]:
del mapid['image'] 

In [45]:
mapid

{u'mapid': u'a29da5a4f9ae49a9c8d7914e75accdf5',
 u'token': u'7c92f8df226df535aa0b0a082c829e7d'}

In [46]:
im = str(mapid['mapid'])

In [50]:
tk = str(mapid['token'])
tk

'7c92f8df226df535aa0b0a082c829e7d'

In [76]:
nomeCountry = 'Benin'
justCountry = countries.filter(ee.Filter.eq('Country', nomeCountry))
raster_tagliato = WPbm.clip(justCountry)
cutPoly = justCountry.geometry()
cutBoundingBox = cutPoly.bounds(1)

regione_paese_json = ee.Geometry(cutPoly.getInfo()).toGeoJSONString()
#regione_paese_json

regione_json = cutPoly.getInfo()
#regione_json

In [77]:
mappa_regione = ETaCollection.getRegion(regione_json)

In [83]:
mappa_regione

AttributeError: 'List' object has no attribute 'toList'